In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import json
import requests
import csv
import re

In [2]:
link = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv'

#Retrieve updated csv from USA Factbook
with requests.Session() as session:
    download = session.get(link)
    decodedContent = download.content.decode('utf-8-sig')

    content = csv.reader(decodedContent.splitlines(), delimiter = ',')
    coviddf = pd.DataFrame(content)

coviddf.head()

,0,1,2,3,4,5,6,7,8,9,...,184,185,186,187,188,189,190,191,192,193
0,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
1,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,857,865,886,905,921,932,942,965,974,974
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,2013,2102,2196,2461,2513,2662,2708,2770,2835,2835
4,1005,Barbour County,AL,1,0,0,0,0,0,0,...,503,514,518,534,539,552,562,569,575,575


In [3]:
#Get rid of unnecessary first few rows
coviddf.columns = coviddf.iloc[0]
coviddf = coviddf[1:]
coviddf['countyFIPS'] = coviddf['countyFIPS'].astype(int)
coviddf.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
1,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,857,865,886,905,921,932,942,965,974,974
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,2013,2102,2196,2461,2513,2662,2708,2770,2835,2835
4,1005,Barbour County,AL,1,0,0,0,0,0,0,...,503,514,518,534,539,552,562,569,575,575
5,1007,Bibb County,AL,1,0,0,0,0,0,0,...,279,283,287,289,303,318,324,334,337,338


In [4]:
unprocessed_coviddf = coviddf.copy(deep=False)

In [5]:
#Get rid of statewide unallocated
coviddf = coviddf[coviddf['countyFIPS']!=0]
coviddf.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,857,865,886,905,921,932,942,965,974,974
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,2013,2102,2196,2461,2513,2662,2708,2770,2835,2835
4,1005,Barbour County,AL,1,0,0,0,0,0,0,...,503,514,518,534,539,552,562,569,575,575
5,1007,Bibb County,AL,1,0,0,0,0,0,0,...,279,283,287,289,303,318,324,334,337,338
6,1009,Blount County,AL,1,0,0,0,0,0,0,...,507,524,547,585,615,637,646,669,675,675


In [6]:
#Drop all the dates except the newest one
coviddf.drop(coviddf.iloc[:,3:-1],axis=1,inplace=True)
coviddf.head()

,countyFIPS,County Name,State,7/29/20
2,1001,Autauga County,AL,974
3,1003,Baldwin County,AL,2835
4,1005,Barbour County,AL,575
5,1007,Bibb County,AL,338
6,1009,Blount County,AL,675


In [7]:
#Get population data
popdf = pd.read_csv('../data/us_popdata_2019.csv')

#Condense pop. data
popdf = popdf.loc[:,['STATE','COUNTY','CTYNAME','POPESTIMATE2019']]
#Add leading zeros
popdf['COUNTY'] = popdf['COUNTY'].apply(lambda x: str(x).zfill(3))
popdf.head()

,STATE,COUNTY,CTYNAME,POPESTIMATE2019
0,1,000,Alabama,4903185
1,1,001,Autauga County,55869
2,1,003,Baldwin County,223234
3,1,005,Barbour County,24686
4,1,007,Bibb County,22394


In [8]:
#Combine state and county to fips (county fips must match in both datasets)
popdf.rename(columns={'COUNTY':'fips'},inplace=True)
popdf['fips'] = popdf['STATE'].apply(lambda x: str(x))+popdf['fips']
popdf.drop(labels=['STATE'],axis=1,inplace=True)
popdf['fips']=pd.to_numeric(popdf['fips'])

#Get rid of state levels
popdf = popdf.loc[popdf['fips']%1000!=0]
popdf.head()

,fips,CTYNAME,POPESTIMATE2019
1,1001,Autauga County,55869
2,1003,Baldwin County,223234
3,1005,Barbour County,24686
4,1007,Bibb County,22394
5,1009,Blount County,57826


In [9]:
#Get difference of counties
print(list(set(coviddf['countyFIPS'])-set(popdf['fips'])))

[6000, 1, 2158, 46102]


In [10]:
#Kusilvak and Wade Hampton are the same, get rid of Kusilvak to match GeoJSON
coviddf = coviddf.loc[coviddf['County Name']!='Kusilvak Census Area']

In [11]:
#Oglala Lakota County in the GeoJSON file is Shannon County (name was changed in 2015)
coviddf.loc[coviddf['County Name']=='Oglala Lakota County', 'countyFIPS'] = 46113
coviddf.loc[coviddf['countyFIPS']==46113, 'County Name'] = 'Shannon County'

In [12]:
popdf.rename(columns={'fips':'countyFIPS'},inplace=True)
popdf.head()

,countyFIPS,CTYNAME,POPESTIMATE2019
1,1001,Autauga County,55869
2,1003,Baldwin County,223234
3,1005,Barbour County,24686
4,1007,Bibb County,22394
5,1009,Blount County,57826


In [13]:
#Merge population/Covid data
coviddf = pd.merge(left = coviddf, right = popdf, on = 'countyFIPS', how = 'outer')
coviddf.head()

,countyFIPS,County Name,State,7/29/20,CTYNAME,POPESTIMATE2019
0,1001,Autauga County,AL,974,Autauga County,55869.0
1,1003,Baldwin County,AL,2835,Baldwin County,223234.0
2,1005,Barbour County,AL,575,Barbour County,24686.0
3,1007,Bibb County,AL,338,Bibb County,22394.0
4,1009,Blount County,AL,675,Blount County,57826.0


In [14]:
#Drop counties that are not officially recognized (Princess Cruise Ship, NYC, etc.)
coviddf = coviddf[coviddf['CTYNAME'].notna()]

In [15]:
#Drop unneeded columns
coviddf.drop(labels=['CTYNAME','State'], axis=1, inplace=True,)
coviddf.head()

,countyFIPS,County Name,7/29/20,POPESTIMATE2019
0,1001,Autauga County,974,55869.0
1,1003,Baldwin County,2835,223234.0
2,1005,Barbour County,575,24686.0
3,1007,Bibb County,338,22394.0
4,1009,Blount County,675,57826.0


In [16]:
#Rename most recent date column and convert columns to int
date = str(coviddf.columns.to_list()[-2])
coviddf.rename(columns={coviddf.columns[-2]:'cases'},inplace=True)
coviddf['cases'] = coviddf['cases'].astype(float)
coviddf['POPESTIMATE2019'] = coviddf['POPESTIMATE2019'].astype(float)
coviddf.head()


,countyFIPS,County Name,cases,POPESTIMATE2019
0,1001,Autauga County,974.0,55869.0
1,1003,Baldwin County,2835.0,223234.0
2,1005,Barbour County,575.0,24686.0
3,1007,Bibb County,338.0,22394.0
4,1009,Blount County,675.0,57826.0


In [17]:
#Add cases per 100k
coviddf['cases_per_100k'] = coviddf['cases'].div(coviddf['POPESTIMATE2019']).mul(100000)
coviddf.head()

,countyFIPS,County Name,cases,POPESTIMATE2019,cases_per_100k
0,1001,Autauga County,974.0,55869.0,1743.363941
1,1003,Baldwin County,2835.0,223234.0,1269.967836
2,1005,Barbour County,575.0,24686.0,2329.255448
3,1007,Bibb County,338.0,22394.0,1509.332857
4,1009,Blount County,675.0,57826.0,1167.294988


In [18]:
#Format to GeoJSON id
coviddf['countyFIPS']="0500000US"+coviddf['countyFIPS'].apply(lambda x: str(x).zfill(5))
coviddf.head()

,countyFIPS,County Name,cases,POPESTIMATE2019,cases_per_100k
0,0500000US01001,Autauga County,974.0,55869.0,1743.363941
1,0500000US01003,Baldwin County,2835.0,223234.0,1269.967836
2,0500000US01005,Barbour County,575.0,24686.0,2329.255448
3,0500000US01007,Bibb County,338.0,22394.0,1509.332857
4,0500000US01009,Blount County,675.0,57826.0,1167.294988


In [19]:
#Rename for a better naming system
coviddf.rename(columns={'countyFIPS':'county_fips','County Name':'county_name','POPESTIMATE2019':'population'}, inplace=True)
coviddf.head()

,county_fips,county_name,cases,population,cases_per_100k
0,0500000US01001,Autauga County,974.0,55869.0,1743.363941
1,0500000US01003,Baldwin County,2835.0,223234.0,1269.967836
2,0500000US01005,Barbour County,575.0,24686.0,2329.255448
3,0500000US01007,Bibb County,338.0,22394.0,1509.332857
4,0500000US01009,Blount County,675.0,57826.0,1167.294988


In [20]:
#Format date for file output name
date_components = re.split('(\W)', date)
for i in np.arange(0, len(date_components), 2):
    date_components[i] = date_components[i].zfill(2)
date_components = ['_' if x=='/' else x for x in date_components]
date = ''.join(date_components)


In [21]:
#Export processed and unprocessed dataframes
coviddf.to_csv('../output/Processed - USCovidbyCounty_'+date+'.csv',index=False)
unprocessed_coviddf.to_csv('../output/Unprocessed - USCovidbyCounty_'+date+'.csv',index=False)

In [ ]:
#Export choropleth map

uscounties = '../data/uscounties.json'

usmap = json.load(open(uscounties))

fig1=px.choropleth_mapbox(data_frame=coviddf,
                    geojson=usmap,
                    featureidkey='properties.GEO_ID',
                    locations='county_fips',
                    color='cases_per_100k',
                    color_continuous_scale='Viridis',
                    zoom=3,
                    opacity=0.7,
                    #Colours range from minimum of dataset to 99th percentile
                    range_color=(coviddf['cases_per_100k'].min(), np.percentile(coviddf['cases_per_100k'].tolist(), 99)),
                    mapbox_style='carto-positron',
                    hover_name='county_name')

fig1.write_html('../output/uscoviddistribution_'+date+'.html')